In [1]:
import os
import tempfile

from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from dotenv import load_dotenv
import pinecone # noqa
import tiktoken
from PyPDF2 import PdfReader

C:\Users\torre\Documents\Dev\chat-with-docs\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()

True

Lectura de pdf y conversion a un solo texto

In [19]:
docs = []
temp_dir = tempfile.TemporaryDirectory()
folder = 'data'
texto = ""

for file in os.listdir(folder):
    path_file = os.path.join(folder, file)
    reader = PdfReader(path_file)
    
    texto = "--------------------------\n"
    texto += "INICIO DEL DOCUMENTO\n"
    texto += "TITULO: " + file.replace("_", " ").replace(".pdf", " ") + '\n'
    
    for pagina in reader.pages:
        texto += pagina.extract_text() + '\n'
        
    texto += "FIN DEL DOCUMENTO\n"
    texto += "--------------------------\n"
    
    with open(f'text/{file.replace(".pdf", ".txt")}', 'w', encoding='utf-8') as archivo_txt:
        archivo_txt.write(texto)
        
# Crear loaders
folder = 'text'

for file in os.listdir(folder):
    print(file)
    
    file_path = os.path.join(folder, file)
    loader = TextLoader(file_path, encoding='utf-8')
    docs.extend(loader.load())

CONSTITUCIÓN_POLÍTICA_DE_LA_REPÚBLICA.txt
porpuesta_de_nueva_constitucion_2022.txt
porpuesta_de_nueva_constitucion_2023.txt


Lectura de pdf y creación de loader

In [6]:
docs = []
temp_dir = tempfile.TemporaryDirectory()
folder = 'data'

for file in os.listdir(folder):
    print(file)
    
    file_path = os.path.join(folder, file)
    loader = PyPDFLoader(file_path=file_path)

    docs.extend(loader.load_and_split())

CONSTITUCIÓN_POLÍTICA_DE_LA_REPÚBLICA.pdf
porpuesta_de_nueva_constitucion_2022.pdf
porpuesta_de_nueva_constitucion_2023.pdf


In [9]:
len(docs)

545

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=500)
splits = text_splitter.split_documents(docs)

In [22]:
len(splits)

605

In [23]:
def consultar_costo_embedding(p):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in p])
    print(f"Total tokens {total_tokens}")
    print(f"Costo embedding {total_tokens / 1000 * 0.0001:.6f}")

In [24]:
consultar_costo_embedding(splits)

Total tokens 406465
Costo embedding 0.040647


In [25]:
api_key = os.getenv('OPENAI_API_KEY')
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [26]:
api_pinecone = os.getenv('PINECONE_API_KEY')
env_pinecone = os.getenv('PINECONE_ENV')

pinecone.init(
    api_key=api_pinecone,
    environment=env_pinecone
)
index_name = "constitution-idx"


In [27]:
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, metric="cosine", dimension=1536)

# docsearch = Pinecone.from_text([t.page_content for t in pages], embeddings, index_name=index_name)
# docsearch = Pinecone.from_documents(pages, embeddings, index_name=index_name)
docsearch = Pinecone.from_documents(splits, embeddings, index_name=index_name)

# docsearch = Pinecone.from_existing_index(index_name, embeddings)

index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 605}},
 'total_vector_count': 605}

In [18]:
docsearch.similarity_search(
    "propuesta nueva constitución",
    k=7
)

[Document(page_content='--------------------------\nINICIO DEL DOCUMENTO\nTITULO: porpuesta de nueva constitucion 2022', metadata={'source': 'text\\porpuesta_de_nueva_constitucion_2022.txt'}),
 Document(page_content='--------------------------\nINICIO DEL DOCUMENTO\nTITULO: porpuesta de nueva constitucion 2023 \nCONSTITUCIÓN POLÍTICAPROPUESTA \nDE LA REPÚBLICA DE CHILE\n2\nEsta propuesta de Constitución Política de la República fue elaborada y \naprobada por el Consejo Constitucional, elegido el día 7 de mayo de 2023, \na partir del anteproyecto elaborado por la Comisión Experta, nombrada por \nla Cámara de Diputadas y Diputados y el Senado.\nEl Consejo Constitucional se instaló el día 7 de junio de 2023 y desarrolló \nsu trabajo desde esa fecha y hasta el día 7 de noviembre de 2023. \nPropuesta Constitución Política de la República de Chile3\nEl Consejo estuvo integrado por:\nAlihuen Antileo Navarrete\nKaren Araya Rojas\nMarcela Araya Sepúlveda\nAntonio Barchiesi Chávez\nGermán Becker

In [ ]:
# retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 4})